# Clase: Gestión e Imputación de Valores Nulos

## 1. Introducción a los Valores Nulos 

### ¿Qué son los Valores Nulos?
En el mundo del análisis de datos, es común encontrarse con **datos incompletos o valores faltantes**. Estos pueden ser el resultado de errores en la recopilación o simplemente la ausencia de información. Saber cómo manejarlos es esencial para análisis precisos y obtener *insights* significativos.

### Tipos de Valores Nulos
Los valores nulos pueden presentarse de diferentes formas en los datos:
*   **`NaN` (Not a Number):** Común en columnas numéricas y de tipo objeto en Pandas, representado por `np.nan`.
*   **`None`:** Una constante de Python, no numérica, que se encuentra en columnas de tipo objeto.
*   **Valores especiales de datos:** Como "NA" o "N/A" en encuestas.
*   **Valores vacíos o en blanco:** Cadenas vacías o espacios en blanco en columnas de texto.
*   **Valores centinelas:** Valores especiales como `-1` o `999` usados como marcadores de nulos.
*   **`NaT` (Not a Timestamp):** Para fechas y horas faltantes.

### ¿Por qué es Importante Gestionar los Valores Nulos?
La gestión de valores nulos es crucial por varias razones:
*   **Evitar errores en el análisis:** Pueden causar problemas en cálculos y análisis estadísticos, distorsionando conclusiones.
*   **Mantener la integridad de los datos:** Asegura la integridad y evita la pérdida de información valiosa.
*   **Mejorar la precisión de los modelos de aprendizaje automático:** Muchos algoritmos no pueden manejarlos, requiriendo su gestión antes del entrenamiento.
*   **Optimizar el uso de recursos:** Ocupan espacio y pueden ralentizar el procesamiento.

### Principales Causas de Valores Faltantes
Las causas pueden variar e influir en la estrategia de manejo:
*   **Errores en la recopilación de datos:** Transcripción o problemas técnicos.
*   **Falta de respuesta:** En encuestas o estudios.
*   **Datos incompletos:** Recopilación parcial de variables o grupos.
*   **Fallos en la transmisión de datos:** Pérdida durante la transferencia.

### Estrategias para Gestionar Valores Nulos (Introducción)
Las estrategias comunes incluyen:
1.  **Eliminación de datos:** Borrar filas o columnas con nulos (`drop()`, `dropna()`).
2.  **Imputación de valores:** Reemplazar los nulos con valores estimados (estadísticos o modelos).
    *   Herramientas clave: **Pandas** (`fillna()`) y **Scikit-learn** (`SimpleImputer`, `KNNImputer`, `IterativeImputer`).

## 2. Preparación y Identificación Inicial de Nulos 

Antes de sumergirnos en la imputación, necesitamos configurar nuestro entorno y entender dónde están los nulos.



In [1]:
# 1. Instalación de librerías (descomentar y ejecutar si no están instaladas)
# !pip install scikit-learn
# !pip install seaborn
# !pip install matplotlib

# 2. Importación de librerías necesarias
# Para tratamiento de datos
import pandas as pd 
import numpy as np 

# Para imputación de nulos usando métodos estadísticos avanzados
from sklearn.impute import SimpleImputer 
from sklearn.experimental import enable_iterative_imputer # Necesario para IterativeImputer
from sklearn.impute import IterativeImputer 
from sklearn.impute import KNNImputer 

# Para visualización
# import seaborn as sns 
# import matplotlib.pyplot as plt 

# 3. Configuración de Pandas para visualizar todas las columnas
pd.set_option('display.max_columns', None) 

In [2]:
# 4. Carga del DataFrame
# Usaremos 'bank-additional_clean.csv' para empezar la gestión de nulos.
df = pd.read_csv("data/bank-additional_clean.csv", index_col=0)
df.head(1)

,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,empvarrate,conspriceidx,consconfidx,euribor3m,nremployed,y,date,latitude,longitude,contact_month,contact_year,age_cat
0,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079,NaN,housemaid,married,basic 4y,No,No,No,telephone,261,1,NaN,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no,2-agosto-2019,41.495,-71.233,agosto,2019.0,Adultos mayores


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43000 entries, 0 to 42999
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   income             43000 non-null  int64  
 1   kidhome            43000 non-null  int64  
 2   teenhome           43000 non-null  int64  
 3   dt_customer        43000 non-null  object 
 4   numwebvisitsmonth  43000 non-null  int64  
 5   id                 43000 non-null  object 
 6   age                37880 non-null  float64
 7   job                42655 non-null  object 
 8   marital            42915 non-null  object 
 9   education          41193 non-null  object 
 10  default            34019 non-null  object 
 11  housing            41974 non-null  object 
 12  loan               41974 non-null  object 
 13  contact            43000 non-null  object 
 14  duration           43000 non-null  int64  
 15  campaign           43000 non-null  int64  
 16  pdays              1588 non

In [ ]:
# 5. Cálculo del porcentaje de nulos por columna
df.isnull().sum()

income                   0
kidhome                  0
teenhome                 0
dt_customer              0
numwebvisitsmonth        0
id                       0
age                   5120
job                    345
marital                 85
education             1807
default               8981
housing               1026
loan                  1026
contact                  0
duration                 0
campaign                 0
pdays                41412
previous                 0
poutcome                 0
empvarrate               0
conspriceidx           471
consconfidx              0
euribor3m             9256
nremployed               0
y                        0
date                   248
latitude                 0
longitude                0
contact_month          248
contact_year           248
age_cat                  0
dtype: int64

In [6]:
df.shape

(43000, 31)

In [7]:
# Dividimos la suma de valores nulos entre el número de filas del df y multiplicamos por 100 para sacar el porcentaje
(df.isnull().sum() / df.shape[0]) * 100

income                0.000000
kidhome               0.000000
teenhome              0.000000
dt_customer           0.000000
numwebvisitsmonth     0.000000
id                    0.000000
age                  11.906977
job                   0.802326
marital               0.197674
education             4.202326
default              20.886047
housing               2.386047
loan                  2.386047
contact               0.000000
duration              0.000000
campaign              0.000000
pdays                96.306977
previous              0.000000
poutcome              0.000000
empvarrate            0.000000
conspriceidx          1.095349
consconfidx           0.000000
euribor3m            21.525581
nremployed            0.000000
y                     0.000000
date                  0.576744
latitude              0.000000
longitude             0.000000
contact_month         0.576744
contact_year          0.576744
age_cat               0.000000
dtype: float64

In [13]:
# Redondeamos, ordenamos
round((df.isnull().sum() / df.shape[0]) * 100, 2).sort_values(ascending=False)

pdays                96.31
euribor3m            21.53
default              20.89
age                  11.91
education             4.20
housing               2.39
loan                  2.39
conspriceidx          1.10
job                   0.80
contact_year          0.58
contact_month         0.58
date                  0.58
marital               0.20
nremployed            0.00
income                0.00
y                     0.00
latitude              0.00
longitude             0.00
consconfidx           0.00
campaign              0.00
empvarrate            0.00
poutcome              0.00
previous              0.00
kidhome               0.00
duration              0.00
contact               0.00
id                    0.00
numwebvisitsmonth     0.00
dt_customer           0.00
teenhome              0.00
age_cat               0.00
dtype: float64

In [14]:
# Otra manera de hacer lo mismo
porc_nulos = (df.isnull().sum() / df.shape[0]) * 100
df_nulos = pd.DataFrame(porc_nulos, columns = ["%_nulos"])
df_nulos = round(df_nulos.sort_values(by="%_nulos", ascending=False), 2)
df_nulos

,%_nulos
pdays,96.31
euribor3m,21.53
default,20.89
age,11.91
education,4.20
housing,2.39
loan,2.39
conspriceidx,1.10
job,0.80
contact_year,0.58


In [15]:
# Filtramos para mostrar solo las columnas con nulos
df_nulos[df_nulos["%_nulos"] > 0]

,%_nulos
pdays,96.31
euribor3m,21.53
default,20.89
age,11.91
education,4.20
housing,2.39
loan,2.39
conspriceidx,1.10
job,0.80
contact_year,0.58




**Nota sobre la eliminación de columnas:** No hay un porcentaje fijo para decidir si eliminar una columna debido a nulos. La decisión **dependerá del contexto y el análisis**. Algunos expertos sugieren que un porcentaje mayor al **5% o 10%** del total de datos puede ser considerado "grande" y requerir atención especial.

## 3. Imputación de Valores Nulos en Variables Categóricas 

Para variables categóricas, podemos usar dos enfoques principales:
*   **Imputación basada en la moda:** Reemplazar nulos con el valor más frecuente. Útil si hay una categoría dominante y queremos mantener la distribución original.
*   **Imputación como una categoría especial:** Mantener los nulos como una nueva categoría (ej., "Unknown"). Útil si no hay una categoría dominante o si queremos distinguir los valores imputados.

La elección depende del conocimiento del dominio y del propósito del análisis.



In [16]:
# 1. Obtener la lista de columnas categóricas con nulos
df.select_dtypes(include='O').columns # Ver las columnas del df que sean de tipo 'object'

Index(['dt_customer', 'id', 'job', 'marital', 'education', 'default',
       'housing', 'loan', 'contact', 'poutcome', 'nremployed', 'y', 'date',
       'contact_month', 'age_cat'],
      dtype='object')

In [17]:
df.select_dtypes(include='O').columns.to_list() # to_list() convierte el Index a lista 

['dt_customer',
 'id',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'poutcome',
 'nremployed',
 'y',
 'date',
 'contact_month',
 'age_cat']

In [21]:
# Creamos una variable nueva para guardar el resultado de pasar la lista de columnas de tipo objeto al df, aquí estamos filtrando. 
serie_nulos = df[df.select_dtypes(include='O').columns.to_list()].isnull().sum() # Además sobre el resultado hacemos isnull y sum
serie_nulos # Lo que nos devuelve una Series con la suma de los valores nulos de las columnas categóricas

dt_customer         0
id                  0
job               345
marital            85
education        1807
default          8981
housing          1026
loan             1026
contact             0
poutcome            0
nremployed          0
y                   0
date              248
contact_month     248
age_cat             0
dtype: int64

In [23]:
serie_nulos[serie_nulos > 0].index # Filtramos la Series por los valores que sean mayores a 0 y sacamos los indices

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'date',
       'contact_month'],
      dtype='object')

In [24]:
# Otra manera de hacer lo mismo
nulos_esta_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
nulos_esta_cat

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'date',
       'contact_month'],
      dtype='object')

In [33]:
# 2. Analizar la distribución de categorías para cada columna con nulos
print("\nDistribución de las categorías para las columnas categóricas con nulos:")
for col in serie_nulos[serie_nulos > 0].index:
    print(f"\nLa distribución de las categorías para la columna {col.upper()}")
    display(df[col].value_counts(normalize=True, dropna=False)*100) # Muestra el porcentaje de cada categoría y el porcentaje de nulos
    print("........................")


Distribución de las categorías para las columnas categóricas con nulos:

La distribución de las categorías para la columna JOB


job
admin.           25.286047
blue-collar      22.451163
technician       16.339535
services          9.679070
management        7.093023
retired           4.162791
entrepreneur      3.539535
self-employed     3.462791
housemaid         2.611628
unemployed        2.472093
student           2.100000
NaN               0.802326
Name: proportion, dtype: float64

........................

La distribución de las categorías para la columna MARITAL


marital
married     60.462791
single      28.151163
divorced    11.188372
NaN          0.197674
Name: proportion, dtype: float64

........................

La distribución de las categorías para la columna EDUCATION


...

........................

La distribución de las categorías para la columna DATE


date
NaN                  0.576744
4-septiembre-2019    0.095349
4-agosto-2017        0.090698
22-junio-2019        0.088372
10-abril-2018        0.086047
                       ...   
19-agosto-2015       0.025581
21-diciembre-2019    0.023256
4-abril-2017         0.023256
20-noviembre-2015    0.023256
14-abril-2015        0.023256
Name: proportion, Length: 1861, dtype: float64

........................

La distribución de las categorías para la columna CONTACT_MONTH


contact_month
noviembre     8.379070
octubre       8.369767
julio         8.355814
marzo         8.346512
abril         8.330233
febrero       8.318605
septiembre    8.300000
mayo          8.248837
junio         8.227907
agosto        8.200000
enero         8.179070
diciembre     8.167442
NaN           0.576744
Name: proportion, dtype: float64

........................


In [46]:
# 3. Decidir la estrategia de imputación para cada columna categórica (basado en el análisis de 'value_counts()')
# Basado en la dominancia de categorías, las columnas a reemplazar por la moda (categoría dominante) son:
# 'marital' (60% married), 'default' (79% No), 'loan' (82% No)
columnas_moda = ["marital", "loan", "default"]

# Columnas a reemplazar por una nueva categoría "Unknown" (no hay categoría dominante):
# 'job', 'education', 'housing', 'contact_month', 'date'
columnas_desconocido = ["job", "education", "housing", "contact_month", "date"]

In [42]:
df['marital'].mode()

0    married
Name: marital, dtype: object

In [43]:
df.marital.mode().iloc[0]

'married'

In [44]:
df.marital.mode()[0]

'married'

In [47]:
# 4. Imputación usando el método .fillna() de Pandas
# El método fillna() se usa para rellenar valores nulos (NaN) con un valor específico.

# a. Reemplazar por la moda para las columnas designadas
print("\n--- Imputando columnas por la moda ---")
for columna in columnas_moda:
    df[columna] = df[columna].fillna(df[columna].mode().iloc[0]) # Rellena los nulos con la moda calculada
    print(f"Columna '{columna}': Nulos reemplazados por la moda '{df[columna].mode().iloc[0]}'")


--- Imputando columnas por la moda ---
Columna 'marital': Nulos reemplazados por la moda 'married'
Columna 'loan': Nulos reemplazados por la moda 'No'
Columna 'default': Nulos reemplazados por la moda 'No'


In [48]:
# b. Comprobar nulos después de la imputación por la moda
print("\nNulos restantes después de reemplazar por la moda:")
df[columnas_moda].isnull().sum()


Nulos restantes después de reemplazar por la moda:


marital    0
loan       0
default    0
dtype: int64

In [49]:
# c. Reemplazar por la categoría "Unknown" para las columnas designadas
print("\n--- Imputando columnas con 'Unknown' ---")
for columna in columnas_desconocido:
    df[columna] = df[columna].fillna("Unknown") # Rellena los nulos con la cadena "Unknown"
    print(f"Columna '{columna}': Nulos reemplazados por 'Unknown'")


--- Imputando columnas con 'Unknown' ---
Columna 'job': Nulos reemplazados por 'Unknown'
Columna 'education': Nulos reemplazados por 'Unknown'
Columna 'housing': Nulos reemplazados por 'Unknown'
Columna 'contact_month': Nulos reemplazados por 'Unknown'
Columna 'date': Nulos reemplazados por 'Unknown'


In [50]:
# d. Comprobar nulos después de la imputación con "Unknown"
print("\nNulos restantes después de reemplazar por 'Unknown':")
df[columnas_desconocido].isnull().sum()


Nulos restantes después de reemplazar por 'Unknown':


job              0
education        0
housing          0
contact_month    0
date             0
dtype: int64

In [51]:
# 5. Guardar el DataFrame con los nulos categóricos imputados
df.to_csv("data/bank-additional_clean_2.csv", index=False)
print("\nDataFrame guardado como 'bank-additional_clean_2.csv' con nulos categóricos imputados.")


DataFrame guardado como 'bank-additional_clean_2.csv' con nulos categóricos imputados.




## 4. Imputación de Valores Nulos en Variables Numéricas 

Para variables numéricas, tenemos más opciones:
*   **Reemplazar con estadísticos:** Media, mediana o moda (`fillna()` de Pandas o `SimpleImputer` de Scikit-learn).
*   **Reemplazar con modelos estadísticos avanzados:** `IterativeImputer` o `KNNImputer`.

**Criterios de decisión para variables numéricas**:
1.  **Cantidad de valores faltantes:** Pequeña cantidad (`<5-10%`): media o mediana. Grande: métodos más sofisticados.
2.  **Distribución de los datos:** Si no hay valores atípicos (outliers): media. Si hay valores atípicos: mediana es más robusta.
3.  **Propósito del análisis:** Si el análisis se centra en la detección de atípicos, es preferible usar métodos que preserven la variabilidad (`IterativeImputer`, `KNNImputer`).



In [52]:
# 1. Obtener la lista de columnas numéricas con nulos
nulos_esta_num = df[df.columns[df.isnull().any()]].select_dtypes(include = np.number).columns
nulos_esta_num

Index(['age', 'pdays', 'conspriceidx', 'euribor3m', 'contact_year'], dtype='object')

In [53]:
# 2. Calcular el porcentaje de nulos para estas columnas numéricas
print("\nPorcentaje de nulos en columnas numéricas:")
(df[nulos_esta_num].isnull().sum() / df.shape[0])*100


Porcentaje de nulos en columnas numéricas:


age             11.906977
pdays           96.306977
conspriceidx     1.095349
euribor3m       21.525581
contact_year     0.576744
dtype: float64

### 🔎 ¿Qué es un outlier?

Los outliers (o valores atípicos) son datos que se alejan mucho del resto de los valores de una columna. Son extremos, inusuales, y pueden deberse a errores, situaciones excepcionales o variabilidad natural. Detectarlos es importante porque pueden influir en los análisis y resultados, distorsionando las estadísticas, como la media.

---

### 🧒 Ejemplo 1: Edades de un grupo de alumnas

Supón que tenemos las edades de un grupo de alumnas:

```python
edades = [15, 16, 17, 16, 18, 17, 16, 72]
```

La mayoría de edades están entre 15 y 18 años, pero hay una persona con **72 años**, que es un outlier.

```python
import numpy as np

np.mean(edades)   # Media
np.median(edades) # Mediana
```

**Resultados:**

* Media: \~23.375
* Mediana: 16.5

🔍 **Explicación**: La media sube mucho debido al 72, pero la mediana se mantiene cerca del centro real del grupo. Por eso decimos que la mediana **es más robusta ante outliers**.

---

### 💸 Ejemplo 2: Salarios de un grupo

Supón estos salarios en miles de euros:

```python
salarios = [25, 26, 24, 25, 27, 26, 25, 1000]
```

La mayoría gana entre 24 k€ y 27 k€, pero hay una persona con un sueldo **de 1 millón** (1000 k€).

```python
np.mean(salarios)   # Media
np.median(salarios) # Mediana
```

**Resultados:**

* Media: \~147.25 k€
* Mediana: 25.5 k€

🔍 **Explicación**: La media se ve fuertemente afectada por el sueldo millonario, mientras que la mediana sigue representando bien al grupo. Nuevamente, la mediana es más fiable cuando hay outliers.

---

### ✅ Conclusión

En presencia de outliers, la **mediana es una mejor medida del centro de los datos** que la media, porque no se ve tan afectada por los valores extremos.

In [ ]:
# 5. Decidir estrategia de imputación para cada columna numérica:
# 'conspriceidx' y 'contact_year': Porcentaje bajo de nulos (<5-10%), sin outliers para 'conspriceidx'.
#   - 'conspriceidx': usaremos .fillna()
#   - 'contact_year': usaremos SimpleImputer (Ambos reemplazan por media/mediana/moda, la diferencia es solo en el código)
# 'age', 'pdays', 'euribor3m': Tienen outliers ('age', 'pdays') o alto porcentaje de nulos.
#   - Usaremos IterativeImputer y KNNImputer para comparar.

In [63]:
df["conspriceidx"].mean() # Calcula la media

93.57421926215054

In [64]:
# 6. Imputación de 'conspriceidx' usando .fillna() con la media
print("\n--- Imputación de 'conspriceidx' con fillna() (media) ---")
media_conspriceidx = df["conspriceidx"].mean() # Calcula la media
print(f"La media de la columna 'conspriceidx' es: {round(media_conspriceidx, 2)}")
df["conspriceidx"] = df["conspriceidx"].fillna(media_conspriceidx) # Rellena los nulos
print(f"Después del 'fillna' tenemos {df['conspriceidx'].isnull().sum()} nulos en 'conspriceidx'")


--- Imputación de 'conspriceidx' con fillna() (media) ---
La media de la columna 'conspriceidx' es: 93.57
Después del 'fillna' tenemos 0 nulos en 'conspriceidx'


In [65]:
# 7. Imputación de 'contact_year' usando SimpleImputer
print("\n--- Imputación de 'contact_year' con SimpleImputer ---")
# SimpleImputer reemplaza valores faltantes con estrategias como media, mediana, moda o un valor constante.

# ¿Media o Mediana?
# - Media: Promedio, útil si los datos están distribuidos uniformemente y sin outliers.
# - Mediana: Valor central, robusta ante outliers o datos sesgados.
# Podemos saber si los datos están sesgados comparando media y mediana; grandes diferencias sugieren sesgo.
print("\nEstadísticos de 'contact_year' para decidir entre media y mediana:")
display(df["contact_year"].describe()[["mean", "50%"]]) # '50%' es la mediana

# En este caso, la media y la mediana son muy parecidas, por lo que usaremos la media.
imputer_contact_year = SimpleImputer(strategy = "mean") # Crea una instancia del imputer con estrategia 'mean'
# Ajustar y transformar los datos. fit_transform espera un array 2D, por eso los dos corchetes.
contact_year_imputado = imputer_contact_year.fit_transform(df[["contact_year"]])

# Sobreescribir la columna original con los valores imputados
df["contact_year"] = contact_year_imputado
print(f"Después del 'SimpleImputer' tenemos {df['contact_year'].isnull().sum()} nulos en 'contact_year'")


--- Imputación de 'contact_year' con SimpleImputer ---

Estadísticos de 'contact_year' para decidir entre media y mediana:


mean    2017.00131
50%     2017.00000
Name: contact_year, dtype: float64

Después del 'SimpleImputer' tenemos 0 nulos en 'contact_year'


In [66]:
# 8. Imputación de 'age', 'pdays', 'euribor3m' con IterativeImputer y KNNImputer
print("\n--- Imputación con métodos avanzados (IterativeImputer y KNNImputer) ---")
# Haremos una copia del DataFrame para aplicar ambos métodos y comparar.
df_copia = df.copy()

# a. IterativeImputer
# Utiliza un modelo de regresión para estimar valores faltantes basándose en otras columnas.
# max_iter: número de iteraciones. random_state: para reproducibilidad.
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)
# Ajustar y transformar los datos para las columnas especificadas.
imputer_iterative_imputado = imputer_iterative.fit_transform(df_copia[["age", "pdays", "euribor3m"]])

# Añadir las columnas imputadas al DataFrame copia con nuevos nombres
df_copia[["age_iterative", "pdays_iterative", "euribor_iterative"]] = imputer_iterative_imputado
print(f"Después del 'IterativeImputer' tenemos: \n{df_copia[['age_iterative', 'pdays_iterative', 'euribor_iterative']].isnull().sum()} nulos") 

# b. KNNImputer
# Basado en el algoritmo de k-vecinos más cercanos (KNN). Estima valores faltantes usando observaciones similares.
# n_neighbors: número de vecinos a considerar.
print("\nPreparando KNNImputer (puede tardar un poco)...")
imputer_knn = KNNImputer(n_neighbors = 5) 
# Ajustar y transformar los datos para las mismas columnas.
imputer_knn_imputado = imputer_knn.fit_transform(df_copia[["age", "pdays", "euribor3m"]]) 

# Añadir las columnas imputadas al DataFrame copia con nuevos nombres
df_copia[["age_knn", "pdays_knn", "euribor_knn"]] = imputer_knn_imputado 
print(f"Después del 'KNNImputer' tenemos: \n{df_copia[['age_knn', 'pdays_knn', 'euribor_knn']].isnull().sum()} nulos") 


--- Imputación con métodos avanzados (IterativeImputer y KNNImputer) ---
Después del 'IterativeImputer' tenemos: 
age_iterative        0
pdays_iterative      0
euribor_iterative    0
dtype: int64 nulos

Preparando KNNImputer (puede tardar un poco)...
Después del 'KNNImputer' tenemos: 
age_knn        0
pdays_knn      0
euribor_knn    0
dtype: int64 nulos


In [67]:
# 9. Comparar los resultados de IterativeImputer y KNNImputer
print("\n--- Comparando estadísticos de las columnas originales y las imputadas ---")
# Compararemos los estadísticos (media, mediana, etc.) antes y después de la imputación.
# Nos quedaremos con el método que menos modifique los estadísticos.
df_copia.describe()[["age","age_iterative", "age_knn",
                          "pdays", "pdays_iterative", "pdays_knn",
                          "euribor3m", "euribor_iterative", "euribor_knn"]]

# Análisis de los resultados:
# - 'age': Ni la media ni la mediana cambian mucho. Ambos métodos son correctos.
# - 'pdays': El IterativeImputer cambia mucho la media y la mediana. KNNImputer es más apropiado.
# - 'euribor3m': Similar a 'age', ambos métodos son válidos.


--- Comparando estadísticos de las columnas originales y las imputadas ---


,age,age_iterative,age_knn,pdays,pdays_iterative,pdays_knn,euribor3m,euribor_iterative,euribor_knn
count,37880.000000,43000.000000,43000.000000,1588.000000,43000.000000,43000.000000,33744.000000,43000.000000,43000.000000
mean,39.977112,39.988040,39.934414,6.072418,2.650482,6.123526,3.616521,3.595413,3.624018
std,10.437957,9.933618,9.921883,3.863182,2.330438,2.030092,1.737117,1.567801,1.604428
min,17.000000,-50.903756,17.000000,0.000000,-0.577994,0.000000,0.634000,-12.678706,0.634000
25%,32.000000,33.000000,32.200000,3.000000,0.907566,4.600000,1.344000,1.445000,1.415000
50%,38.000000,39.955946,39.000000,6.000000,1.828094,5.800000,4.857000,4.076000,4.731800
75%,47.000000,46.000000,46.000000,7.000000,4.000000,7.200000,4.961000,4.959000,4.959000
max,98.000000,98.000000,98.000000,27.000000,27.000000,27.000000,5.045000,5.830842,5.045000


In [68]:
# 10. Limpiar el DataFrame final y guardar
print("\n--- Limpiando el DataFrame final ---")
# Eliminamos las columnas originales con nulos y las imputaciones menos óptimas.
df_copia.drop(["age", "pdays", "euribor3m", # Originales con nulos
              "age_knn" , "euribor_knn", # Podríamos haber elegido cualquiera de los dos, eliminamos estos
              "pdays_iterative" # El que peor imputó para pdays
             ], axis = 1, inplace = True)

# Cambiar el nombre de las columnas restantes a sus nombres originales
nuevo_nombre = {"age_iterative": "age", 'euribor_iterative': "euribor3m", "pdays_knn": "pdays" } 
df_copia.rename(columns = nuevo_nombre, inplace = True) 

# Guardar el DataFrame final sin nulos para futuras lecciones
df_copia.to_csv("data/bank-additional-clean-nonulls.csv", index=False) 
print("\nDataFrame final sin nulos guardado como 'bank-additional-clean-nonulls.csv'.")


--- Limpiando el DataFrame final ---

DataFrame final sin nulos guardado como 'bank-additional-clean-nonulls.csv'.


In [69]:
df_copia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43000 entries, 0 to 42999
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   income             43000 non-null  int64  
 1   kidhome            43000 non-null  int64  
 2   teenhome           43000 non-null  int64  
 3   dt_customer        43000 non-null  object 
 4   numwebvisitsmonth  43000 non-null  int64  
 5   id                 43000 non-null  object 
 6   job                43000 non-null  object 
 7   marital            43000 non-null  object 
 8   education          43000 non-null  object 
 9   default            43000 non-null  object 
 10  housing            43000 non-null  object 
 11  loan               43000 non-null  object 
 12  contact            43000 non-null  object 
 13  duration           43000 non-null  int64  
 14  campaign           43000 non-null  int64  
 15  previous           43000 non-null  int64  
 16  poutcome           43000 no



## 5. Conclusiones

### Consideraciones Clave
*   **Cada conjunto de datos es único** y las decisiones de imputación deben basarse en una comprensión profunda del dominio.
*   Siempre es una buena idea **evaluar el efecto de la imputación** en la distribución y calidad de tus datos antes de continuar con el análisis.
*   La **causa de los valores faltantes** puede influir en la estrategia.

---